# 날씨데이터 정리

날씨 데이터 구조

1. 지역별 : 노원구/종로구
2. 날짜별 : 측정소 별로 측정 시작 시간 다름, 매분마다 측정, 의미 없을듯
3. 위치별 : 측정소 위치별 데이터 수집, 동기화 필요

할 일

1. Data Merge
  1.1) 파일 읽기
  1.2) 날짜 기준으로 파일 정리(노원구 / 종로구 구분해서 먼저)
  1.3) 파일 이름 = 지역 = 위치 
        →파일 이름을 Column으로 추가,  



In [1]:
import pandas as pd
import os
import numpy as np

In [4]:
#데이터 합쳐서 저장하기

file_path = 'C:\\Users\Sunghyun\Google 드라이브\SGkaggler\data\주어진데이터\환경기상데이터\\'
data = pd.DataFrame([])
goo_list = ['종로구','노원구']


for goo in goo_list:
    #print(goo)
    file_list2 = os.listdir('C:\\Users\\Sunghyun\\Desktop\\Blog\\Data Science\\BIG CONTEST\\데이터\\주어진 데이터\\Raw\\환경기상데이터' + '\\' + goo)
    #print(file_list2)
    for file in file_list2:
        filer_path = file_path + '\\' + goo + '\\' + file
        #print(filer_path)
        data_read = pd.read_csv(filer_path, na_values = ('-999', '-9999'))
        #날짜척도 추가하기
        #data_read['시간'] = data_read['tm'] // 100
        data_read['날짜'] = data_read['tm'] // 10000
        #날짜척도로 묶기(날짜평균 계산)
        data_time = data_read.groupby('날짜', as_index = False).mean()
        data_time['serial'] = file[:-4]
        data = pd.concat([data, data_time])

In [5]:
data[data[['flag', 'pm10', 'co2', 'vocs', 'noise', 'temp', 'humi', 'pm25']]<0] = np.nan

In [6]:
data[data['pm10']<0]

,날짜,tm,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10,serial


In [7]:
data.head()

,날짜,tm,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10,serial
0,20180401,2.018040e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468
1,20180402,2.018040e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468
2,20180403,2.018040e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468
3,20180404,2.018040e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468
4,20180405,2.018041e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468


In [8]:
#미세먼지 척도 추가하기
data_raw = data

dt = data['pm10']

data['예보'] = np.where(dt > 150, '매우나쁨', 
                             np.where(dt > 80, '나쁨',
                                     np.where(dt > 30, '보통', '좋음')))

#NaN 값들을 처리하지 못함!!!개선해야함


data.head()

,날짜,tm,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10,serial,예보
0,20180401,2.018040e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468,좋음
1,20180402,2.018040e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468,좋음
2,20180403,2.018040e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468,좋음
3,20180404,2.018040e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468,좋음
4,20180405,2.018041e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,V01o1610468,좋음


In [9]:
#측정 위치 : 구/행정동으로 정리하기

norm_path = 'C:\\Users\Sunghyun\Desktop\Blog\Data Science\BIG CONTEST\데이터\주어진 데이터\Raw\환경기상데이터'

norm_no = norm_path + '\\' + '노원구.csv'
norm_jong = norm_path + '\\' + '종로구.csv'

norm_jong_csv = pd.DataFrame(pd.read_csv(norm_jong, encoding = 'euc-kr')[['스테이션', '행정동']])
norm_jong_csv['구'] = '종로구'

norm_no_csv = pd.DataFrame(pd.read_csv(norm_no, encoding = 'euc-kr')[['스테이션', '행정동']])
norm_no_csv['구'] = '노원구'

norm_merge_csv = pd.concat([norm_no_csv, norm_jong_csv]).reset_index()
norm_merge_csv = norm_merge_csv[['스테이션', '구', '행정동']]
norm_merge_csv = norm_merge_csv.rename(columns = {'스테이션' : 'serial'})
norm_merge_csv.head()


,serial,구,행정동
0,V10O1610610,노원구,하계1동
1,V10O1610376,노원구,"상계6,7동"
2,V10O1610293,노원구,상계10동
3,V10O1610356,노원구,상계1동
4,V10O1610616,노원구,상계2동


In [10]:
#데이터 합체(분단위 데이터)

data = data.merge(norm_merge_csv, on = 'serial')

data.tail()

,날짜,tm,flag,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10,serial,예보,구,행정동
16100,20190327,2.019033e+11,1.0,105.061806,NaN,NaN,54.225694,12.733194,56.488194,68.551389,NaN,V10O1612126,나쁨,노원구,중계본동
16101,20190328,2.019033e+11,1.0,92.713194,NaN,NaN,54.228472,11.268819,42.620833,58.077083,NaN,V10O1612126,나쁨,노원구,중계본동
16102,20190329,2.019033e+11,1.0,45.061111,NaN,NaN,53.993750,10.795625,24.930556,23.551389,NaN,V10O1612126,보통,노원구,중계본동
16103,20190330,2.019033e+11,1.0,52.343750,NaN,NaN,55.382639,6.358750,66.666667,38.128472,NaN,V10O1612126,보통,노원구,중계본동
16104,20190331,2.019033e+11,1.0,48.583333,NaN,NaN,54.609028,5.752917,42.414583,34.352778,NaN,V10O1612126,보통,노원구,중계본동


In [11]:
data_cols = data.columns.tolist()
data_cols = [data_cols[-4]] + [data_cols[0]] + data_cols[-2:] + [data_cols[-3]] + data_cols[3:11]

data = data[data_cols]

data.head()


,serial,날짜,구,행정동,예보,pm10,co2,vocs,noise,temp,humi,pm25,Unnamed: 10
0,V01o1610468,20180401,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V01o1610468,20180402,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V01o1610468,20180403,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V01o1610468,20180404,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V01o1610468,20180405,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data = data[data_cols[:-1]]

data.head()

,serial,날짜,구,행정동,예보,pm10,co2,vocs,noise,temp,humi,pm25
0,V01o1610468,20180401,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V01o1610468,20180402,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V01o1610468,20180403,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,V01o1610468,20180404,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,V01o1610468,20180405,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#data['시'] = data['시간']%100
#data['날짜'] = pd.to_datetime(data['시간']//100, format = '%Y%m%d')

#data.head()

In [14]:
del(data['시간'])

KeyError: '시간'

In [15]:
del(data['serial'])

In [21]:
data = data.rename(columns = {'행정동': '동'})

In [22]:
data.head()

,날짜,구,동,예보,pm10,co2,vocs,noise,temp,humi,pm25
0,20180401,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20180402,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20180403,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20180404,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20180405,종로구,사직동,좋음,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
data.to_csv('환경기상데이터.csv', encoding='euc-kr', index = None)